<a href="https://colab.research.google.com/github/razreshili/esg-embedding-model/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libraries

In [ ]:
# Install necessary libraries
! pip install datasets
! pip install huggingface_hub
! pip install -U sentence-transformers
! pip install peft
! pip install wandb
# Connect to hugging face and wandb
! huggingface-cli login --token YOUR_HF_TOKEN
! wandb login YOUR_WANDB_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.7 MB/s eta 0:00:00
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from datasets import load_dataset

from sentence_transformers import SentenceTransformer, SentenceTransformerTrainingArguments, SentenceTransformerTrainer
from transformers import EarlyStoppingCallback
from sentence_transformers.losses import MultipleNegativesRankingLoss, TripletLoss
from sentence_transformers.training_args import BatchSamplers # to remove duplicates while training
from sentence_transformers.evaluation import TripletEvaluator, SequentialEvaluator, SentenceEvaluator

from peft import LoraConfig, IA3Config, get_peft_model

import os
import logging

# For custom evaluator
import torch.nn.functional as F
import torch
import csv
from contextlib import nullcontext

In [ ]:
os.environ["WANDB_PROJECT"]="esg-embeddings-compare"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

# Load triplet dataset

In [ ]:
#dataset = load_dataset("esg-x/esg-triplet-dataset", token=True)
#dataset = load_dataset("esg-x/triplet-dataset", token=True)
dataset = load_dataset("esg-x/esg-triplet-adjacent-dataset", token=True)
#dataset = load_dataset("esg-x/triplet-dataset-with-hard-negatives", token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/60.8M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/20.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62292 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/20764 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20764 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.remove_columns("doc_id")
#dataset = dataset.remove_columns("negative")

In [ ]:
dataset = dataset.remove_columns("pairs_unique_id")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 62292
    })
    val: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 20764
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 20764
    })
})

In [ ]:
train_dataset = dataset["train"]
eval_dataset = dataset["val"]
test_dataset = dataset["test"]

# Model

In [ ]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#model = SentenceTransformer("nreimers/albert-small-v2", device = "cuda")

## LoRA

In [ ]:
for param in model.parameters():
    param.requires_grad = False

config = LoraConfig(
    target_modules = ["value", "query"],
    use_dora=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.01,
    bias="none",
)

peft_config = IA3Config(
    target_modules=["key", "value", "intermediate.dense"], feedforward_modules=["intermediate.dense"]
)
model = get_peft_model(model, config)

### Check number of trainable params

In [ ]:
model.print_trainable_parameters()

trainable params: 147,456 || all params: 22,860,672 || trainable%: 0.6450


In [ ]:
for name, param in model.named_parameters():
    print(f"Name: {name}, Size: {param.size()}, Requires Grad: {param.requires_grad}")

Name: base_model.model.0.auto_model.embeddings.word_embeddings.weight, Size: torch.Size([30522, 384]), Requires Grad: False
Name: base_model.model.0.auto_model.embeddings.position_embeddings.weight, Size: torch.Size([512, 384]), Requires Grad: False
Name: base_model.model.0.auto_model.embeddings.token_type_embeddings.weight, Size: torch.Size([2, 384]), Requires Grad: False
Name: base_model.model.0.auto_model.embeddings.LayerNorm.weight, Size: torch.Size([384]), Requires Grad: False
Name: base_model.model.0.auto_model.embeddings.LayerNorm.bias, Size: torch.Size([384]), Requires Grad: False
Name: base_model.model.0.auto_model.encoder.layer.0.attention.self.query.base_layer.weight, Size: torch.Size([384, 384]), Requires Grad: False
Name: base_model.model.0.auto_model.encoder.layer.0.attention.self.query.base_layer.bias, Size: torch.Size([384]), Requires Grad: False
Name: base_model.model.0.auto_model.encoder.layer.0.attention.self.query.lora_A.default.weight, Size: torch.Size([16, 384]), 

In [ ]:
model

PeftModel(
  (base_model): LoraModel(
    (model): SentenceTransformer(
      (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
      (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      (2): Normalize()
    )
  )
)

# Custom evaluator

In [ ]:
logger = logging.getLogger(__name__)

In [ ]:
class UniformityAlignmentEvaluator(SentenceEvaluator):
    """
    Evaluator for Alignment and Uniformity metrics
    """

    def __init__(
        self,
        anchors: list[str],
        positives: list[str],
        negatives: list[str],
        name: str = "",
        batch_size: int = 16,
        show_progress_bar: bool = True,
        write_csv: bool = True,
        truncate_dim: int | None = None,
    ):
        super().__init__()
        self.anchors = anchors
        self.positives = positives
        self.negatives = negatives
        self.name = name
        self.truncate_dim = truncate_dim

        assert len(self.anchors) == len(self.positives)
        assert len(self.anchors) == len(self.negatives)

        self.batch_size = batch_size
        if show_progress_bar is None:
            show_progress_bar = (
                logger.getEffectiveLevel() == logging.INFO or logger.getEffectiveLevel() == logging.DEBUG
            )
        self.show_progress_bar = show_progress_bar
        self.write_csv = write_csv
        self.primary_metric = "alignment"

        self.csv_file = "uniformity_alignment_evaluation" + ("_" + name if name else "") + "_results.csv"
        self.csv_headers = ["epoch", "steps", "alignment", "uniformity", "alignment_negative"]

    def align_loss(self, x, y, alpha=2):
        return (x - y).norm(p=2, dim=1).pow(alpha).mean()

    def uniform_loss(self, x, t=2):
        return torch.pdist(x, p=2).pow(2).mul(-t).exp().mean().log()

    def calculate_alignment_and_uniformity(self, embeddings_anchors, embeddings_positives, embeddings_negatives):
        # Normalize embeddings before computing alignment and uniformity
        embeddings_anchors = F.normalize(embeddings_anchors, dim=-1)
        embeddings_positives = F.normalize(embeddings_positives, dim=-1)
        embeddings_negatives = F.normalize(embeddings_negatives, dim=-1)

        # Compute alignment
        alignment = self.align_loss(embeddings_anchors, embeddings_positives)
        alignment_negative = self.align_loss(embeddings_anchors, embeddings_negatives)

        # Compute uniformity for each embedding set
        uniform_anchors = self.uniform_loss(embeddings_anchors)
        uniform_positives = self.uniform_loss(embeddings_positives)
        uniform_negatives = self.uniform_loss(embeddings_negatives)

        # Compute average uniformity (similar to how it's done in calculate_loss example)
        uniformity = (uniform_anchors + uniform_positives + uniform_negatives) / 3

        return alignment, alignment_negative, uniformity

    def calculate_accuracy_cos(self, embeddings_anchors, embeddings_positives, embeddings_negatives):
        # Cosine similarity (higher is more similar)
        pos_cos_similarity = F.cosine_similarity(embeddings_anchors, embeddings_positives)
        neg_cos_similarity = F.cosine_similarity(embeddings_anchors, embeddings_negatives)

        # Count triplet accuracy based on cosine similarity
        num_triplets = len(pos_cos_similarity)
        num_correct_cos_triplets = torch.sum(pos_cos_similarity > neg_cos_similarity).item()

        accuracy_cos = num_correct_cos_triplets / num_triplets
        return accuracy_cos

    def __call__(self, model: SentenceTransformer, output_path: str = None, epoch: int = -1, steps: int = -1) -> dict[str, float]:
        if epoch != -1:
            if steps == -1:
                out_txt = f" after epoch {epoch}"
            else:
                out_txt = f" in epoch {epoch} after {steps} steps"
        else:
            out_txt = ""
        if self.truncate_dim is not None:
            out_txt += f" (truncated to {self.truncate_dim})"

        logger.info(f"UniformityAlignmentEvaluator: Evaluating the model on the {self.name} dataset{out_txt}:")

        with nullcontext() if self.truncate_dim is None else model.truncate_sentence_embeddings(self.truncate_dim):
            embeddings_anchors = model.encode(
                self.anchors,
                batch_size=self.batch_size,
                show_progress_bar=self.show_progress_bar,
                convert_to_tensor=True,
            ).to("cuda")
            embeddings_positives = model.encode(
                self.positives,
                batch_size=self.batch_size,
                show_progress_bar=self.show_progress_bar,
                convert_to_tensor=True,
            ).to("cuda")
            embeddings_negatives = model.encode(
                self.negatives,
                batch_size=self.batch_size,
                show_progress_bar=self.show_progress_bar,
                convert_to_tensor=True,
            ).to("cuda")
        # Compute accuracy based on cosine similarity
        accuracy_cos = self.calculate_accuracy_cos(embeddings_anchors, embeddings_positives, embeddings_negatives)
        # Compute alignment and uniformity
        alignment, alignment_negative, uniformity = self.calculate_alignment_and_uniformity(
            embeddings_anchors, embeddings_positives, embeddings_negatives
        )
        logger.info(f"Alignment: {alignment:.6f}")
        logger.info(f"Uniformity: {uniformity:.6f}")
        logger.info(f"Alignment negative: {alignment_negative:.6f}")
        logger.info(f"Accuracy Cosine Distance:   \t{accuracy_cos * 100:.2f}%")

        if output_path is not None and self.write_csv:
            csv_path = os.path.join(output_path, self.csv_file)
            if not os.path.isfile(csv_path):
                with open(csv_path, newline="", mode="w", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(self.csv_headers)
                    writer.writerow([epoch, steps, alignment, uniformity, alignment_negative, accuracy_cos])

            else:
                with open(csv_path, newline="", mode="a", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow([epoch, steps, alignment, uniformity, alignment_negative, accuracy_cos])

        metrics = {
            "alignment": float(alignment),
            "uniformity": float(uniformity),
            "alignment_negative": float(alignment_negative),
            "accuracy_cos": float(accuracy_cos),
        }
        metrics = self.prefix_name_to_metrics(metrics, self.name)
        self.store_metrics_in_model_card_data(model, metrics)
        return metrics

    @property
    def description(self) -> str:
        return "Uniformity and Alignment Evaluation"


# Train

InfoNCE loss is synonomuos with MultipleNegativesRankingLoss (source: https://github.com/UKPLab/sentence-transformers/issues/1305 )

In [ ]:
# 4. Define a loss function
#loss = TripletLoss(model)
loss = MultipleNegativesRankingLoss(model)

In [ ]:
# 5. (Optional) Specify training arguments
# check documentation to change params
# https://sbert.net/docs/package_reference/sentence_transformer/training_args.html
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/esg-embedding",
    # Optional training parameters:
    num_train_epochs=20, # initial is 5; try 50 later
    per_device_train_batch_size=64, #later 128 good perform
    per_device_eval_batch_size=512, #128
    learning_rate=1e-4, # initial is 2e-5; 2e-3 good perform
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="epoch", # was steps
    #eval_steps=500,
    save_strategy="epoch", # was steps
    #load_best_model_at_end=True,
    #save_steps=500,
    save_total_limit=2,
    logging_steps=500, # may make higher, defines how often to log the perform
    run_name="lora-adjacent",  # wandb, name of the current run
    seed=42
)

In [ ]:
# Initialize early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,  # Stop if the validation loss does not improve for 3 epochs
    early_stopping_threshold=0.0  # Minimum change to consider improvement
)

### Checking evaluator

In [ ]:
dev_evaluator = UniformityAlignmentEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="esg-embedding-dev",
)

In [ ]:
#dev_evaluator(model)

# Run

In [ ]:
# 7. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator
    #callbacks=[early_stopping]
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arina-zemchik (esg-x-ml). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss,Esg-embedding-dev Alignment,Esg-embedding-dev Uniformity,Esg-embedding-dev Alignment Negative,Esg-embedding-dev Accuracy Cos
1,1.808000,2.638577,1.140972,-3.711898,1.900847,0.975294
2,1.360500,2.419409,1.102480,-3.736951,1.917922,0.980640
3,1.233700,2.308656,1.087107,-3.755028,1.928951,0.983240
4,1.159400,2.237475,1.079442,-3.773033,1.939238,0.984830
5,1.100800,2.169766,1.070115,-3.790298,1.947900,0.985696
6,1.056300,2.129538,1.057034,-3.790580,1.950141,0.986660
7,1.025800,2.098322,1.054318,-3.797781,1.954230,0.986515
8,0.997400,2.074689,1.049883,-3.800923,1.956551,0.987430
9,0.978600,2.050702,1.046776,-3.809025,1.960960,0.987430
10,0.955100,2.035625,1.047361,-3.813581,1.962842,0.987960


Batches:   0%|          | 0/1298 [00:00<?, ?it/s]

Batches:   0%|          | 0/1298 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1298 [00:00<?, ?it/s]

Batches:   0%|          | 0/1298 [00:00<?, ?it/s]

Batches:   0%|          | 0/1298 [00:00<?, ?it/s]

wandb: Adding directory to artifact (./models/esg-embedding/checkpoint-19480)... Done. 0.0s
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


TrainOutput(global_step=19480, training_loss=1.0129452981498452, metrics={'train_runtime': 3931.9304, 'train_samples_per_second': 316.852, 'train_steps_per_second': 4.954, 'total_flos': 0.0, 'train_loss': 1.0129452981498452, 'epoch': 20.0})

In [ ]:
# (Optional) Evaluate the trained model on the test set
#test_evaluator = UniformityAlignmentEvaluator(
#    anchors=test_dataset["anchor"],
#    positives=test_dataset["positive"],
#    negatives=test_dataset["negative"],
#    name="esg-embedding-test",
#)
#test_evaluator(model)

# Downstream performance

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
base_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
dataset = load_dataset("FinanceMTEB/ESG")
train_dataset = dataset["train"]
test_dataset = dataset["test"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Extract text and labels
train_texts = train_dataset['text']
train_labels = train_dataset['label']
test_texts = test_dataset['text']
test_labels = test_dataset['label']
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
def train_and_print(model):
  # Generate embeddings using GPU and then convert them to CPU for scikit-learn
  train_embeddings = model.encode(train_texts, convert_to_tensor=True, device=device).cpu().numpy()
  test_embeddings = model.encode(test_texts, convert_to_tensor=True, device=device).cpu().numpy()
  # Train a simple Logistic Regression model
  clf = LogisticRegression(max_iter=100, multi_class='multinomial', solver='lbfgs')
  #clf = RidgeClassifier()
  #clf = GaussianNB()
  #clf = DecisionTreeClassifier()
  #clf = KNeighborsClassifier()
  #clf = RandomForestClassifier()
  #clf = svm.SVC()
  clf.fit(train_embeddings, train_labels)

  # Make predictions on the test set
  test_predictions = clf.predict(test_embeddings)

  # Evaluate the classifier
  print(classification_report(test_labels, test_predictions))

README.md:   0%|          | 0.00/441 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/325k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/111k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_and_print(model)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94       190
           1       0.88      0.57      0.69        37
           2       0.95      0.96      0.96       497
           3       0.90      0.93      0.91       276

    accuracy                           0.93      1000
   macro avg       0.92      0.85      0.87      1000
weighted avg       0.93      0.93      0.93      1000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
train_and_print(base_model)

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       190
           1       0.96      0.59      0.73        37
           2       0.94      0.96      0.95       497
           3       0.89      0.93      0.91       276

    accuracy                           0.93      1000
   macro avg       0.94      0.85      0.88      1000
weighted avg       0.93      0.93      0.93      1000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
